# **Tutorial** - (Custom) Embedding Models in BERTopic
(last updated 11-09-2022)

In this tutorial we will be going through the embedding models that can be used in BERTopic. Having the option to choose embedding models allow you to leverage pre-trained embeddings that suit your use-case. Moreover, it helps creating a topic when you have little data to your availability. 

## Embedding models
Embedding models are used for the representation of text, such as words, sentences and documents, in the form of real-valued vectors. They typically encode the semantic meaning of text. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic, with all backends possible, from PyPi:

In [6]:
%%capture
!pip install bertopic[flair, gensim, spacy, use]

**NOTE**: This may take a while as it needs to install Spacy, Torch, Gensim, USE, etc. 

**NOTE 1**: There might be dependency-conflicts if you install back-ends so it might be worthwhile to only choose one to experiment with. 

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [2]:
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups
docs = fetch_20newsgroups(subset='train',  remove=('headers', 'footers', 'quotes'))['data']

In [3]:
print(docs[0])

I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.


# **Embedding Models**
In this section, we will go through all embedding models and backends that are supported in BERTopic.

## Sentence Transformers
You can select any model from sentence-transformers [here](https://www.sbert.net/docs/pretrained_models.html) and pass it through BERTopic with `embedding_model`:

In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens").fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

Or we can select a SentenceTransformer model with our own parameters:



In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("xlm-r-bert-base-nli-stsb-mean-tokens", device="cuda")
topic_model = BERTopic(embedding_model=sentence_model).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

## 🤗 Transformers

To use a Hugging Face transformers model, load in a pipeline and point to any model found on their model hub (https://huggingface.co/models):

In [ ]:
from transformers.pipelines import pipeline

embedding_model = pipeline("feature-extraction", model="distilbert-base-cased")
topic_model = BERTopic(embedding_model=embedding_model).fit(docs)

## Flair

Flair allows you to choose almost any embedding model that is publicly available.<br> Flair can be used as follows:

In [ ]:
from flair.embeddings import TransformerDocumentEmbeddings

roberta = TransformerDocumentEmbeddings('roberta-base')
topic_model = BERTopic(embedding_model=roberta).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

You can select any 🤗 transformers model [here](https://huggingface.co/models).

Moreover, you can also use Flair to use word embeddings and pool them to create document embeddings. Under the hood, Flair simply averages all word embeddings in a document. Then, we can easily pass it to BERTopic in order to use those word embeddings as document embeddings:

In [ ]:
from flair.embeddings import WordEmbeddings, DocumentPoolEmbeddings

glove_embedding = WordEmbeddings('crawl')
document_glove_embeddings = DocumentPoolEmbeddings([glove_embedding])

topic_model = BERTopic(embedding_model=document_glove_embeddings).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

## Spacy
Spacy has shown great promise over the last years and is now slowly transitioning into transformer-based techniques which makes it interesting to use in BERTopic. 

We start by using a non-transformer-based model which we will have to download first:

In [ ]:
!python -m spacy download en_core_web_md

Next, simply load the model into a Spacy nlp instance and pass it through BERTopic:

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md", exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

topic_model = BERTopic(embedding_model=nlp, verbose=True).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

We can also use their transformer-based models which we also have to download first:

In [ ]:
!python -m spacy download en_core_web_trf

As before, we simply load the model and pass it through BERTopic. Note that we exclude a bunch of features as they are not used in BERTopic.

In [ ]:
import spacy

spacy.prefer_gpu()
nlp = spacy.load("en_core_web_trf", exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])
topic_model = BERTopic(embedding_model=nlp, verbose=True, min_topic_size=5).fit(docs)

If you run into memory issues with spacy-transformer models, try:

In [ ]:
import spacy
from thinc.api import set_gpu_allocator, require_gpu

nlp = spacy.load("en_core_web_trf", exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])
set_gpu_allocator("pytorch")
require_gpu(0)

topic_model = BERTopic(embedding_model=nlp, verbose=True).fit(docs)

## Universal Sentence Encoder (USE)
The Universal Sentence Encoder encodes text into high dimensional vectors that are used here for embedding the documents. The model is trained and optimized for greater-than-word length text, such as sentences, phrases or short paragraphs.



In [ ]:
import tensorflow_hub
embedding_model = tensorflow_hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
topic_model = BERTopic(verbose=True, embedding_model=embedding_model).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

## Gensim
For Gensim, BERTopic supports its `gensim.downloader` module. Here, we can download any model word embedding model to be used in BERTopic. Note that Gensim is primarily used for Word Embedding models. This works typically best for short documents since the word embeddings are pooled. 

In [ ]:
import gensim.downloader as api
ft = api.load('fasttext-wiki-news-subwords-300')
topic_model = BERTopic(verbose=True, embedding_model=ft).fit(docs)

# **Customization**
Over the last years, many new embedding models have been released that could be interesting to use as a backend in BERTopic. It is not always feasible to implement them all as there are simply too many to follow. 

In order to still allow to use those embeddings, BERTopic knows several ways to add these embeddings while still allowing for full functionality of BERTopic. 

Moreover, there are several customization options that allow for a bit more control over which embedding to use when. 

## Word + Document Embeddings
You might want to be using different language models for creating document- and word-embeddings.
For example, while SentenceTransformers might be great in embedding sentences and documents, you
might prefer to use FastText to create the word embeddings.

In [ ]:
from bertopic.backend import WordDocEmbedder
import gensim.downloader as api
from sentence_transformers import SentenceTransformer

# Word embedding model
ft = api.load('fasttext-wiki-news-subwords-300')

# Document embedding model
distilbert = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

# Create a model that uses both language models and pass it through BERTopic
word_doc_embedder = WordDocEmbedder(embedding_model=distilbert, word_embedding_model=ft)
topic_model = BERTopic(verbose=True, embedding_model=word_doc_embedder).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

## Custom Backend
If your backend or model cannot be found in the ones currently available, you can use the BaseEmbedder
class to create your own backend. Below, you will find an example of creating a SentenceTransformer backend for BERTopic:


In [ ]:
from bertopic.backend import BaseEmbedder
from sentence_transformers import SentenceTransformer

class CustomEmbedder(BaseEmbedder):
    def __init__(self, embedding_model):
        super().__init__()
        self.embedding_model = embedding_model

    def embed(self, documents, verbose=False):
        embeddings = self.embedding_model.encode(documents, show_progress_bar=verbose)
        return embeddings 

# Create custom backend
distilbert = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")
custom_embedder = CustomEmbedder(embedding_model=distilbert)

# Pass custom backend to bertopic
topic_model = BERTopic(embedding_model=custom_embedder).fit(docs)

In [ ]:
topic_model.get_topic_info().head(5)

## Custom Embeddings
You can use any embedding that you have previously created. This can be to speed up creating topic models but also if your language model does not fit in any of the options above. 

Here, we will be using **TF-IDF** as the main embedder in BERTopic:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

embeddings = TfidfVectorizer(min_df=5, stop_words="english").fit_transform(docs)
topic_model = BERTopic().fit(docs, embeddings)